In [50]:
import os,glob
import numpy as np
import pandas as pd
import glob
import pyslha
import time
import progressbar as P
from atlas_susy_2016_08_Recast import getLLPs, getJets,eventAcc,vertexAcc
from ATLAS_data.effFunctions import vertexEff, eventEff
import matplotlib.pyplot as plt
import os,glob
import numpy as np
import seaborn as sns
from scipy.special import kn,zetac
from helper import LLP
import pyslha
sns.set() #Set style
sns.set_style('ticks',{'font.family':'serif', 'font.serif':'Times New Roman'})
sns.set_context('paper', font_scale=1.8)
sns.set_palette(sns.color_palette("Paired"))

delphesDir = os.path.abspath("../ATLAS-SUSY-2018-13/DelphesLLP")
os.environ['ROOT_INCLUDE_PATH'] = os.path.join(delphesDir,"external")

import ROOT

ROOT.gSystem.Load(os.path.join(delphesDir,"libDelphes.so"))

ROOT.gInterpreter.Declare('#include "classes/SortableObject.h"')
ROOT.gInterpreter.Declare('#include "classes/DelphesClasses.h"')
ROOT.gInterpreter.Declare('#include "external/ExRootAnalysis/ExRootTreeReader.h"')


True

In [51]:
inputFile = './validation/pp2GoGo/Events/run_06/tag_1_delphes_events.root'

In [52]:
f = ROOT.TFile(inputFile,'read')
tree = f.Get("Delphes")
nevts = tree.GetEntries()

ievt = 1

tree.GetEntry(ievt)
for illp in range(tree.llps.GetEntries()):
    llp = tree.llps.At(illp)
try:
    llps = getLLPs(tree.llps,tree.llpDaughters,3e-2)
except:
    print('Error in %i'%ievt)
jets = getJets(tree.GenJet,pTmin=25.,etaMax=5.0)


In [53]:
print(tree.llps.GetEntries())

2


In [54]:
jets = getJets(tree.GenJet,pTmin=25.,etaMax=5.0)
met = tree.DMMissingET.At(0).MET

# Event acceptance
evt_acc = eventAcc(jets,met,metCut=200.0,
                   maxJetChargedPT=5.0,minJetPt1=70.,
                   minJetPt2=25.,minPVdistance=4.0)

llps = getLLPs(tree.llps,tree.llpDaughters)
# Vertex acceptances:
v_acc = np.array([vertexAcc(llp,Rmax=300.0,zmax=300.0,Rmin=4.0,
                            d0min=2.0,nmin=5,mDVmin=10.0)  for llp in llps])
good_llps = np.array(llps)[v_acc > 0.0]

In [55]:
print(len(good_llps))
print(v_acc)

2
[1. 1.]


In [56]:
for llp in llps:
    print(llp.PID,llp.Status)
    print(len(llp.directDaughters),len(llp.finalDaughters))
    print(np.sqrt(llp.Xd**2 + llp.Yd**2))
    print(llp.r_decay,llp.z_decay,llp.mDV,llp.nTracks)
    print('\n')

1000021 106
3 45
62.146097231407474
62.146097231407474 -101.77991485595703 82.82497597766168 12


1000021 106
3 58
198.97048983974395
198.97048983974395 -11.595541954040527 80.08968712908174 14




In [57]:
j = jets[0]

In [65]:
for ip in range(j.Constituents.GetEntries()):
    particle = j.Constituents.At(ip)
    print(particle.PID,particle.Status,particle.PT,partice)

22 1 0.5512533783912659
-211 1 0.2413061410188675
211 1 0.6013044714927673
22 1 0.10199962556362152
22 1 0.11898388713598251
11 1 0.7012597322463989
-11 1 0.3381158411502838
211 1 1.9820307493209839
22 1 0.3153257668018341
-211 1 2.2826859951019287
22 1 4.3992180824279785
22 1 3.045034885406494
-321 1 18.901334762573242
-2112 1 4.82358455657959
22 1 6.622241973876953
211 1 8.116230010986328
-211 1 29.46614646911621
2212 1 91.164794921875
22 1 4.250680923461914
